In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException

In [13]:
# 크롬 옵션즈에 정보를 담아 사람인 것 처럼 만들기
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# 크롬 웹브라우저 드라이버 자동 다운로드
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.set_window_size(1920,1080) # 웹브라우저 해상도 조절
driver.get("https://map.kakao.com/")

In [14]:
try:
    # 해당 요소 찾기
    coach_layer_image = driver.find_element(By.CSS_SELECTOR, "body > div.coach_layer.coach_layer_type1 > div > div > img")
    # JavaScript를 사용해 클릭
    driver.execute_script("arguments[0].click();", coach_layer_image)
    print("코치 레이어 이미지를 클릭했습니다.")
except NoSuchElementException:
    # 요소가 없으면 무시하고 진행
    print("코치 레이어 이미지가 존재하지 않습니다. 다음 단계로 진행합니다.")


코치 레이어 이미지를 클릭했습니다.


In [15]:


# 검색창 찾기
search_box = driver.find_element(By.ID, "search.keyword.query")
search_box.send_keys("뱅뱅막국수")  # 원하는 검색어 입력
search_box.send_keys(Keys.ENTER)  # 엔터 키로 검색 실행

In [16]:
# 잠시 대기 후 첫 번째 검색 결과 클릭
time.sleep(1)  # 검색 결과가 로딩될 시간을 줌
first_result = driver.find_element(By.CSS_SELECTOR, '.placelist .PlaceItem .link_name')  # 첫 번째 항목의 선택자
first_result.click()

In [17]:
# 첫 번째 검색 결과의 리뷰 링크 클릭
moreview_link = driver.find_element(By.CSS_SELECTOR, 'a[data-id="numberofscore"]')
moreview_link.click()

In [18]:
original_window = driver.current_window_handle
for window_handle in driver.window_handles:
    if window_handle != original_window:
        driver.switch_to.window(window_handle)
        break

In [8]:


# '후기 더보기' 버튼을 반복적으로 클릭하여 모든 리뷰를 로드
while True:
    try:
        # '후기 더보기' 버튼이 있는지 확인
        more_reviews_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="mArticle"]/div[7]/div[3]/a'))
        )
        
        # 버튼의 텍스트가 '후기 접기'인지 확인하여 모든 리뷰가 로드되었는지 확인
        if more_reviews_button.text == "후기 접기":
            print("모든 리뷰가 로드되었습니다.")
            break
        
        # '후기 더보기' 버튼 클릭
        driver.execute_script("arguments[0].click();", more_reviews_button)
        print("후기 더보기 버튼을 클릭했습니다.")
        
        # 리뷰가 로드되는 동안 약간의 대기 시간을 추가
        time.sleep(1)

    except TimeoutException:
        # '후기 더보기' 버튼이 없으면 루프 종료
        print("후기 더보기 버튼을 찾을 수 없거나 로드되지 않았습니다. 모든 리뷰가 이미 로드되었거나 버튼이 없습니다.")
        break

    except ElementClickInterceptedException:
        # 버튼이 클릭되지 않는 경우 메시지 출력 후 루프 종료
        print("후기 더보기 버튼을 클릭할 수 없습니다. 다른 요소가 가로막고 있을 수 있습니다.")
        break

print("모든 리뷰 로드 완료.")


후기 더보기 버튼을 클릭했습니다.
후기 더보기 버튼을 클릭했습니다.
모든 리뷰가 로드되었습니다.
모든 리뷰 로드 완료.


In [9]:
# 모든 리뷰를 담을 리스트 초기화
review_dates = []
review_contents = []

# 리뷰 목록 가져오기
review_elements = driver.find_elements(By.CSS_SELECTOR, 'ul.list_evaluation li')

# 각 리뷰에서 날짜와 내용 추출
for review_element in review_elements:
    try:
        # 리뷰 날짜와 내용 추출
        date = review_element.find_element(By.CSS_SELECTOR, '.unit_info .time_write').text
        content = review_element.find_element(By.CSS_SELECTOR, '.txt_comment').text
        
        # 리스트에 저장
        review_dates.append(date)
        review_contents.append(content)
        
    except Exception as e:
        print("리뷰 정보를 가져오는 중 문제가 발생했습니다:", e)

# 데이터프레임으로 저장
reviews_df = pd.DataFrame({
    '작성 날짜': review_dates,
    '리뷰 내용': review_contents
})

# 데이터프레임 출력
print(reviews_df)

리뷰 정보를 가져오는 중 문제가 발생했습니다: Message: no such element: Unable to locate element: {"method":"css selector","selector":".unit_info .time_write"}
  (Session info: chrome=130.0.6723.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x007AEBD3+24307]
	(No symbol) [0x00738D74]
	(No symbol) [0x0061C323]
	(No symbol) [0x0065DC86]
	(No symbol) [0x0065DECB]
	(No symbol) [0x00653CF1]
	(No symbol) [0x0067FED4]
	(No symbol) [0x00653C14]
	(No symbol) [0x00680124]
	(No symbol) [0x00699579]
	(No symbol) [0x0067FC26]
	(No symbol) [0x0065219C]
	(No symbol) [0x0065311D]
	GetHandleVerifier [0x00A58D93+2818227]
	GetHandleVerifier [0x00AB542E+3196750]
	GetHandleVerifier [0x00AAD9D2+3165426]
	GetHandleVerifier [0x0084DA70+675216]
	(No symbol) [0x00741B3D]
	(No symbol) [0x0073EA18]
	(No symbol) [0x0073EBB5]
	(No symbol) [0x00731640]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppCo

리뷰 정보를 가져오는 중 문제가 발생했습니다: Message: no such element: Unable to locate element: {"method":"css selector","selector":".unit_info .time_write"}
  (Session info: chrome=130.0.6723.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x007AEBD3+24307]
	(No symbol) [0x00738D74]
	(No symbol) [0x0061C323]
	(No symbol) [0x0065DC86]
	(No symbol) [0x0065DECB]
	(No symbol) [0x00653CF1]
	(No symbol) [0x0067FED4]
	(No symbol) [0x00653C14]
	(No symbol) [0x00680124]
	(No symbol) [0x00699579]
	(No symbol) [0x0067FC26]
	(No symbol) [0x0065219C]
	(No symbol) [0x0065311D]
	GetHandleVerifier [0x00A58D93+2818227]
	GetHandleVerifier [0x00AB542E+3196750]
	GetHandleVerifier [0x00AAD9D2+3165426]
	GetHandleVerifier [0x0084DA70+675216]
	(No symbol) [0x00741B3D]
	(No symbol) [0x0073EA18]
	(No symbol) [0x0073EBB5]
	(No symbol) [0x00731640]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppCo

In [10]:
reviews_df

,작성 날짜,리뷰 내용
0,2024.10.21.,좋은 모임 자리를 갖게 되어서 기분 좋은 자리 가지고 싶어서 서버분 팁도 먼저 드리...
1,2024.09.11.,
2,2024.02.23.,"가성비가 좋은 건 인정하고 맛도 그럭저럭 좋다. 다만, 룸에서는 일부 메뉴 (등급낮..."
3,2024.01.07.,"주로 따로국밥, 우거지탕과 같은 식사류를 먹어봤는데 가격대비 퀄리티가 너무 좋다. ..."
4,2023.10.06.,
5,2023.09.20.,
6,2023.09.04.,
7,2023.05.21.,
8,2023.05.14.,"진짜 직원들 매너 무엇,,, 기분나빠서 나옴"
9,2023.01.03.,강남에서 높은 질의 한우를 이 가격에 먹을 수 있다는 점에 한 번 놀라고 전라도 김...


In [12]:
reviews_df.to_csv('hangangsu_reviews.csv', index=False, encoding='utf-8-sig')

### -------------------------------------------------------------

In [13]:
# 크롬 옵션즈에 정보를 담아 사람인 것 처럼 만들기
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# 크롬 웹브라우저 드라이버 자동 다운로드
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.set_window_size(1920,1080) # 웹브라우저 해상도 조절
driver.get("https://map.kakao.com/")

In [14]:
try:
    # 해당 요소 찾기
    coach_layer_image = driver.find_element(By.CSS_SELECTOR, "body > div.coach_layer.coach_layer_type1 > div > div > img")
    # JavaScript를 사용해 클릭
    driver.execute_script("arguments[0].click();", coach_layer_image)
    print("코치 레이어 이미지를 클릭했습니다.")
except NoSuchElementException:
    # 요소가 없으면 무시하고 진행
    print("코치 레이어 이미지가 존재하지 않습니다. 다음 단계로 진행합니다.")


코치 레이어 이미지를 클릭했습니다.


In [15]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 검색창 찾기
search_box = driver.find_element(By.ID, "search.keyword.query")
search_box.send_keys("파스쿠찌 뱅뱅사거리")  # 원하는 검색어 입력
search_box.send_keys(Keys.ENTER)  # 엔터 키로 검색 실행

In [16]:
# 잠시 대기 후 첫 번째 검색 결과 클릭
time.sleep(1)  # 검색 결과가 로딩될 시간을 줌
first_result = driver.find_element(By.CSS_SELECTOR, '.placelist .PlaceItem .link_name')  # 첫 번째 항목의 선택자
first_result.click()

In [17]:
# 첫 번째 검색 결과의 리뷰 링크 클릭

moreview_link = driver.find_element(By.CSS_SELECTOR, 'a[data-id="numberofscore"]')
moreview_link.click()

In [18]:
original_window = driver.current_window_handle
for window_handle in driver.window_handles:
    if window_handle != original_window:
        driver.switch_to.window(window_handle)
        break

In [19]:
from selenium.common.exceptions import TimeoutException

# '후기 더보기' 버튼을 반복적으로 클릭하여 모든 리뷰를 로드
while True:
    try:
        # '후기 더보기' 버튼이 있는지 확인
        more_reviews_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="mArticle"]/div[7]/div[3]/a'))
        )
        
        # 버튼의 텍스트가 '후기 접기'인지 확인하여 모든 리뷰가 로드되었는지 확인
        if more_reviews_button.text == "후기 접기":
            print("모든 리뷰가 로드되었습니다.")
            break
        
        # '후기 더보기' 버튼 클릭
        driver.execute_script("arguments[0].click();", more_reviews_button)
        print("후기 더보기 버튼을 클릭했습니다.")
        
        # 리뷰가 로드되는 동안 약간의 대기 시간을 추가
        time.sleep(1)

    except TimeoutException:
        # '후기 더보기' 버튼이 없으면 루프 종료
        print("후기 더보기 버튼을 찾을 수 없거나 로드되지 않았습니다. 모든 리뷰가 이미 로드되었거나 버튼이 없습니다.")
        break

    except ElementClickInterceptedException:
        # 버튼이 클릭되지 않는 경우 메시지 출력 후 루프 종료
        print("후기 더보기 버튼을 클릭할 수 없습니다. 다른 요소가 가로막고 있을 수 있습니다.")
        break

print("모든 리뷰 로드 완료.")


후기 더보기 버튼을 찾을 수 없거나 로드되지 않았습니다. 모든 리뷰가 이미 로드되었거나 버튼이 없습니다.
모든 리뷰 로드 완료.


In [20]:
# 모든 리뷰를 담을 리스트 초기화
review_dates = []
review_contents = []

# 리뷰 목록 가져오기
review_elements = driver.find_elements(By.CSS_SELECTOR, 'ul.list_evaluation li')

# 각 리뷰에서 날짜와 내용 추출
for review_element in review_elements:
    try:
        # 리뷰 날짜와 내용 추출
        date = review_element.find_element(By.CSS_SELECTOR, '.unit_info .time_write').text
        content = review_element.find_element(By.CSS_SELECTOR, '.txt_comment').text
        
        # 리스트에 저장
        review_dates.append(date)
        review_contents.append(content)
        
    except Exception as e:
        print("리뷰 정보를 가져오는 중 문제가 발생했습니다:", e)

# 데이터프레임으로 저장
reviews_df = pd.DataFrame({
    '작성 날짜': review_dates,
    '리뷰 내용': review_contents
})

# 데이터프레임 출력
print(reviews_df)

리뷰 정보를 가져오는 중 문제가 발생했습니다: Message: no such element: Unable to locate element: {"method":"css selector","selector":".unit_info .time_write"}
  (Session info: chrome=130.0.6723.117); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0050EBD3+24307]
	(No symbol) [0x00498D74]
	(No symbol) [0x0037C323]
	(No symbol) [0x003BDC86]
	(No symbol) [0x003BDECB]
	(No symbol) [0x003B3CF1]
	(No symbol) [0x003DFED4]
	(No symbol) [0x003B3C14]
	(No symbol) [0x003E0124]
	(No symbol) [0x003F9579]
	(No symbol) [0x003DFC26]
	(No symbol) [0x003B219C]
	(No symbol) [0x003B311D]
	GetHandleVerifier [0x007B8D93+2818227]
	GetHandleVerifier [0x0081542E+3196750]
	GetHandleVerifier [0x0080D9D2+3165426]
	GetHandleVerifier [0x005ADA70+675216]
	(No symbol) [0x004A1B3D]
	(No symbol) [0x0049EA18]
	(No symbol) [0x0049EBB5]
	(No symbol) [0x00491640]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppCo

In [21]:
reviews_df

,작성 날짜,리뷰 내용
0,2023.09.21.,파스쿠찌 검색하다가 대로가 아닌 골목에 위치해서 놀람. 1층은 거의 좌석없고 2층 ...
1,2019.10.29.,


In [23]:
reviews_df.to_csv('pascucci_reviews.csv', index=False, encoding='utf-8-sig')

## 200m 이내 가게 리뷰 추출

In [10]:
# 크롬 옵션즈에 정보를 담아 사람인 것 처럼 만들기
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# 크롬 웹브라우저 드라이버 자동 다운로드
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.set_window_size(1920,1080) # 웹브라우저 해상도 조절
driver.get("https://map.kakao.com/")

try:
    # 해당 요소 찾기
    coach_layer_image = driver.find_element(By.CSS_SELECTOR, "body > div.coach_layer.coach_layer_type1 > div > div > img")
    # JavaScript를 사용해 클릭
    driver.execute_script("arguments[0].click();", coach_layer_image)
    print("코치 레이어 이미지를 클릭했습니다.")
except NoSuchElementException:
    # 요소가 없으면 무시하고 진행
    print("코치 레이어 이미지가 존재하지 않습니다. 다음 단계로 진행합니다.")
    


# 검색창 찾기
search_box = driver.find_element(By.ID, "search.keyword.query")
search_box.send_keys("스노우폭스 뱅뱅점")  # 원하는 검색어 입력
search_box.send_keys(Keys.ENTER)  # 엔터 키로 검색 실행


# 잠시 대기 후 첫 번째 검색 결과 클릭
time.sleep(1)  # 검색 결과가 로딩될 시간을 줌
first_result = driver.find_element(By.CSS_SELECTOR, '.placelist .PlaceItem .link_name')  # 첫 번째 항목의 선택자
first_result.click()


# 첫 번째 검색 결과의 리뷰 링크 클릭
moreview_link = driver.find_element(By.CSS_SELECTOR, 'a[data-id="numberofscore"]')
moreview_link.click()

original_window = driver.current_window_handle
for window_handle in driver.window_handles:
    if window_handle != original_window:
        driver.switch_to.window(window_handle)
        break
        
        


# '후기 더보기' 버튼을 반복적으로 클릭하여 모든 리뷰를 로드
while True:
    try:
        # '후기 더보기' 버튼이 있는지 확인
        more_reviews_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="mArticle"]/div[7]/div[3]/a'))
        )
        
        # 버튼의 텍스트가 '후기 접기'인지 확인하여 모든 리뷰가 로드되었는지 확인
        if more_reviews_button.text == "후기 접기":
            print("모든 리뷰가 로드되었습니다.")
            break
        
        # '후기 더보기' 버튼 클릭
        driver.execute_script("arguments[0].click();", more_reviews_button)
        print("후기 더보기 버튼을 클릭했습니다.")
        
        # 리뷰가 로드되는 동안 약간의 대기 시간을 추가
        time.sleep(1)

    except TimeoutException:
        # '후기 더보기' 버튼이 없으면 루프 종료
        print("후기 더보기 버튼을 찾을 수 없거나 로드되지 않았습니다. 모든 리뷰가 이미 로드되었거나 버튼이 없습니다.")
        break

    except ElementClickInterceptedException:
        # 버튼이 클릭되지 않는 경우 메시지 출력 후 루프 종료
        print("후기 더보기 버튼을 클릭할 수 없습니다. 다른 요소가 가로막고 있을 수 있습니다.")
        break

print("모든 리뷰 로드 완료.")

# 모든 리뷰를 담을 리스트 초기화
review_dates = []
review_contents = []

# 리뷰 목록 가져오기
review_elements = driver.find_elements(By.CSS_SELECTOR, 'ul.list_evaluation li')

# 각 리뷰에서 날짜와 내용 추출
for review_element in review_elements:
    try:
        # 리뷰 날짜와 내용 추출
        date = review_element.find_element(By.CSS_SELECTOR, '.unit_info .time_write').text
        content = review_element.find_element(By.CSS_SELECTOR, '.txt_comment').text
        
        # 리스트에 저장
        review_dates.append(date)
        review_contents.append(content)
        
    except Exception as e:
        print("리뷰 정보를 가져오는 중 문제가 발생했습니다:", e)

# 데이터프레임으로 저장
reviews_df = pd.DataFrame({
    '작성 날짜': review_dates,
    '리뷰 내용': review_contents
})


reviews_df

코치 레이어 이미지를 클릭했습니다.
후기 더보기 버튼을 클릭했습니다.
후기 더보기 버튼을 클릭했습니다.
모든 리뷰가 로드되었습니다.
모든 리뷰 로드 완료.
리뷰 정보를 가져오는 중 문제가 발생했습니다: Message: no such element: Unable to locate element: {"method":"css selector","selector":".unit_info .time_write"}
  (Session info: chrome=130.0.6723.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x007AEBD3+24307]
	(No symbol) [0x00738D74]
	(No symbol) [0x0061C323]
	(No symbol) [0x0065DC86]
	(No symbol) [0x0065DECB]
	(No symbol) [0x00653CF1]
	(No symbol) [0x0067FED4]
	(No symbol) [0x00653C14]
	(No symbol) [0x00680124]
	(No symbol) [0x00699579]
	(No symbol) [0x0067FC26]
	(No symbol) [0x0065219C]
	(No symbol) [0x0065311D]
	GetHandleVerifier [0x00A58D93+2818227]
	GetHandleVerifier [0x00AB542E+3196750]
	GetHandleVerifier [0x00AAD9D2+3165426]
	GetHandleVerifier [0x0084DA70+675216]
	(No symbol) [0x00741B3D]
	(No symbol) [0x0073EA18]
	(No symbol) [

리뷰 정보를 가져오는 중 문제가 발생했습니다: Message: no such element: Unable to locate element: {"method":"css selector","selector":".unit_info .time_write"}
  (Session info: chrome=130.0.6723.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x007AEBD3+24307]
	(No symbol) [0x00738D74]
	(No symbol) [0x0061C323]
	(No symbol) [0x0065DC86]
	(No symbol) [0x0065DECB]
	(No symbol) [0x00653CF1]
	(No symbol) [0x0067FED4]
	(No symbol) [0x00653C14]
	(No symbol) [0x00680124]
	(No symbol) [0x00699579]
	(No symbol) [0x0067FC26]
	(No symbol) [0x0065219C]
	(No symbol) [0x0065311D]
	GetHandleVerifier [0x00A58D93+2818227]
	GetHandleVerifier [0x00AB542E+3196750]
	GetHandleVerifier [0x00AAD9D2+3165426]
	GetHandleVerifier [0x0084DA70+675216]
	(No symbol) [0x00741B3D]
	(No symbol) [0x0073EA18]
	(No symbol) [0x0073EBB5]
	(No symbol) [0x00731640]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppCo

,작성 날짜,리뷰 내용
0,2024.08.30.,포장으로 한끼 때운다.\n입안에서 퍽퍽하다..
1,2024.04.12.,채소가 물기 하나도없고 텁텁한게 요리할 때 채소를 아예 안씻는거같음. 흙맛나서 다 ...
2,2023.11.06.,옥수수스프를 샀는데 먹을려고 뚜껑 여니까 뭔 젤리처럼 스프가 한 덩어리가 된거임. ...
3,2022.11.09.,먹지마세요… 회사돈으로 먹어도 아까워요\n얼마나 맛있는지 확인차 먹었는데 그 돈으로...
4,2022.08.01.,가성비는 구리지만 양많고 먹을만함. 식사할때 물티슈를 개별포장 제품을 주는 게 아니...
5,2021.09.14.,가성비 별로
6,2021.05.29.,저녁에 가면 도시락이 별로 없어서 아쉬워요
7,2021.05.18.,전반적으로 맛있어요. 근데 가격에 비해 재료를 너무 많이 아꼈단 생각이 드는 곳이에...
8,2020.12.23.,도시락이 여러종류가 있고 깨끗해서 좋긴한데 초밥이 내 입맛에는 짜다. 코우슬로도 짜...
9,2020.04.04.,칼로리 냦고 연어 초밥 계열 & 닭갈비 덮밥 맛있어요


In [12]:
# 크롬 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# 크롬 웹브라우저 드라이버 자동 다운로드
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.set_window_size(1920,1080)
driver.get("https://map.kakao.com/")

try:
    # 코치 레이어 이미지 클릭
    coach_layer_image = driver.find_element(By.CSS_SELECTOR, "body > div.coach_layer.coach_layer_type1 > div > div > img")
    driver.execute_script("arguments[0].click();", coach_layer_image)
    print("코치 레이어 이미지를 클릭했습니다.")
except NoSuchElementException:
    print("코치 레이어 이미지가 존재하지 않습니다. 다음 단계로 진행합니다.")

# 검색창 찾기 및 검색어 입력
search_box = driver.find_element(By.ID, "search.keyword.query")
search_box.send_keys("스노우폭스 뱅뱅점")
search_box.send_keys(Keys.ENTER)

# 첫 번째 검색 결과 클릭
time.sleep(1)
first_result = driver.find_element(By.CSS_SELECTOR, '.placelist .PlaceItem .link_name')
first_result.click()

# 업종 정보 추출
try:
    subcategory = driver.find_element(By.CSS_SELECTOR, 'span[data-id="subcategory"]')
    business_category = subcategory.text
    print("업종 정보를 성공적으로 추출했습니다:", business_category)
except NoSuchElementException:
    business_category = "업종 정보 없음"
    print("업종 정보를 찾을 수 없습니다.")

# 첫 번째 검색 결과의 리뷰 링크 클릭
moreview_link = driver.find_element(By.CSS_SELECTOR, 'a[data-id="numberofscore"]')
moreview_link.click()

# 새로운 창으로 전환
original_window = driver.current_window_handle
for window_handle in driver.window_handles:
    if window_handle != original_window:
        driver.switch_to.window(window_handle)
        break

# '후기 더보기' 버튼을 반복적으로 클릭하여 모든 리뷰를 로드
while True:
    try:
        more_reviews_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="mArticle"]/div[7]/div[3]/a'))
        )
        if more_reviews_button.text == "후기 접기":
            print("모든 리뷰가 로드되었습니다.")
            break
        driver.execute_script("arguments[0].click();", more_reviews_button)
        print("후기 더보기 버튼을 클릭했습니다.")
        time.sleep(1)
    except TimeoutException:
        print("후기 더보기 버튼을 찾을 수 없거나 로드되지 않았습니다. 모든 리뷰가 이미 로드되었거나 버튼이 없습니다.")
        break
    except ElementClickInterceptedException:
        print("후기 더보기 버튼을 클릭할 수 없습니다. 다른 요소가 가로막고 있을 수 있습니다.")
        break

print("모든 리뷰 로드 완료.")

# 리뷰 데이터 초기화
review_dates = []
review_contents = []

# 리뷰 목록 가져오기
review_elements = driver.find_elements(By.CSS_SELECTOR, 'ul.list_evaluation li')
for review_element in review_elements:
    try:
        date = review_element.find_element(By.CSS_SELECTOR, '.unit_info .time_write').text
        content = review_element.find_element(By.CSS_SELECTOR, '.txt_comment').text
        review_dates.append(date)
        review_contents.append(content)
    except Exception as e:
        print("리뷰 정보를 가져오는 중 문제가 발생했습니다:", e)

# 데이터프레임으로 저장
reviews_df = pd.DataFrame({
    '작성 날짜': review_dates,
    '리뷰 내용': review_contents,
    '업종 정보': [business_category] * len(review_dates)
})

reviews_df

코치 레이어 이미지를 클릭했습니다.
업종 정보를 성공적으로 추출했습니다: 도시락
후기 더보기 버튼을 클릭했습니다.
후기 더보기 버튼을 클릭했습니다.
모든 리뷰가 로드되었습니다.
모든 리뷰 로드 완료.
리뷰 정보를 가져오는 중 문제가 발생했습니다: Message: no such element: Unable to locate element: {"method":"css selector","selector":".unit_info .time_write"}
  (Session info: chrome=130.0.6723.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x007AEBD3+24307]
	(No symbol) [0x00738D74]
	(No symbol) [0x0061C323]
	(No symbol) [0x0065DC86]
	(No symbol) [0x0065DECB]
	(No symbol) [0x00653CF1]
	(No symbol) [0x0067FED4]
	(No symbol) [0x00653C14]
	(No symbol) [0x00680124]
	(No symbol) [0x00699579]
	(No symbol) [0x0067FC26]
	(No symbol) [0x0065219C]
	(No symbol) [0x0065311D]
	GetHandleVerifier [0x00A58D93+2818227]
	GetHandleVerifier [0x00AB542E+3196750]
	GetHandleVerifier [0x00AAD9D2+3165426]
	GetHandleVerifier [0x0084DA70+675216]
	(No symbol) [0x00741B3D]
	(No symbol) [0

리뷰 정보를 가져오는 중 문제가 발생했습니다: Message: no such element: Unable to locate element: {"method":"css selector","selector":".unit_info .time_write"}
  (Session info: chrome=130.0.6723.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x007AEBD3+24307]
	(No symbol) [0x00738D74]
	(No symbol) [0x0061C323]
	(No symbol) [0x0065DC86]
	(No symbol) [0x0065DECB]
	(No symbol) [0x00653CF1]
	(No symbol) [0x0067FED4]
	(No symbol) [0x00653C14]
	(No symbol) [0x00680124]
	(No symbol) [0x00699579]
	(No symbol) [0x0067FC26]
	(No symbol) [0x0065219C]
	(No symbol) [0x0065311D]
	GetHandleVerifier [0x00A58D93+2818227]
	GetHandleVerifier [0x00AB542E+3196750]
	GetHandleVerifier [0x00AAD9D2+3165426]
	GetHandleVerifier [0x0084DA70+675216]
	(No symbol) [0x00741B3D]
	(No symbol) [0x0073EA18]
	(No symbol) [0x0073EBB5]
	(No symbol) [0x00731640]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppCo

,작성 날짜,리뷰 내용,업종 정보
0,2024.08.30.,포장으로 한끼 때운다.\n입안에서 퍽퍽하다..,도시락
1,2024.04.12.,채소가 물기 하나도없고 텁텁한게 요리할 때 채소를 아예 안씻는거같음. 흙맛나서 다 ...,도시락
2,2023.11.06.,옥수수스프를 샀는데 먹을려고 뚜껑 여니까 뭔 젤리처럼 스프가 한 덩어리가 된거임. ...,도시락
3,2022.11.09.,먹지마세요… 회사돈으로 먹어도 아까워요\n얼마나 맛있는지 확인차 먹었는데 그 돈으로...,도시락
4,2022.08.01.,가성비는 구리지만 양많고 먹을만함. 식사할때 물티슈를 개별포장 제품을 주는 게 아니...,도시락
5,2021.09.14.,가성비 별로,도시락
6,2021.05.29.,저녁에 가면 도시락이 별로 없어서 아쉬워요,도시락
7,2021.05.18.,전반적으로 맛있어요. 근데 가격에 비해 재료를 너무 많이 아꼈단 생각이 드는 곳이에...,도시락
8,2020.12.23.,도시락이 여러종류가 있고 깨끗해서 좋긴한데 초밥이 내 입맛에는 짜다. 코우슬로도 짜...,도시락
9,2020.04.04.,칼로리 냦고 연어 초밥 계열 & 닭갈비 덮밥 맛있어요,도시락


## 자동화

In [25]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
import pandas as pd
import time

# "오하나 도곡점",
# 가게명 리스트
store_names = ["스노우폭스 뱅뱅점", "맥도날드 서초뱅뱅점", "큐뮬러스", "유니마카롱", "짜짜루", "아티제 강남유니온센터점",
    "어가람", "드시옹", "웰니스 쌀빵", "파리바게뜨 뱅뱅사거리점", "파라노이드 강남뱅뱅 카페", "아티제 뱅뱅사거리점",
    "만복회해산물 뱅뱅사거리점", "해머스미스커피 뱅뱅사거리점", "월참치", "쿡쿡쿡 뱅뱅사거리점", "파스타엔포크 강남역삼점",
    "고가네철판불백", "곰작골나주곰탕 역삼초교점", "사람사는고깃집김일도 뱅뱅점", "잇풀키토김밥 샌드위치 샐러드 강남점",
    "엄마손맛집밥세로방", "고불초쌈밥 2호점", "우심터", "난타5000", "봉구스밥버거 역삼삼일점", "세로방", "커피콩스토리", "더에이치",
    "9CAFE", "소뭉집 본점", "리첸", "지우네", "중화카츠", "교토일식", "회춘식당", "노브랜드버거 뱅뱅사거리점", "라망드쉐프", "서초골",
    "에슬로우커피 뱅뱅사거리점", "전주피순대",
"오로스트커피", "다슬음", "조순금닭도리탕", "궁민김밥", "더벤티 뱅뱅사거리점",
    "돼지벅스", "시원한대구탕", "판문점부대찌개 역삼점", "청담배짱이", "통돼지 두루치기김치찌개", "봉평착한메밀", "칸꼬시",
    "곰바로곰탕", "문화시민", "오늘은 어떤닭", "카페블랑131", "버드나무집 서초본점", "파스쿠찌 뱅뱅사거리점", "남강매점",
    "스타벅스 서초태우빌딩점", "맥켄치킨", "봉피양 양재점", "깡돈", "북경반점", "무화잠 강남점", "아워948", "참족", "예당", "다돈식당",
    "낙여삼", "명동할머니국수 뱅뱅사거리점", "이천쌀밥", "또봉이통닭 도곡점", "카린지린가네스낵바 뱅뱅사거리점", "왕산골",
    "서초남순남순대국 본점", "브라운돈까스 뱅뱅사거리점", "고품격커피공장 뱅뱅사거리점", "두두돼지불백", "사천루",
 "장수촌풍천장어",
    "뱅뱅막국수 역삼본점", "커피박스", "뚜레쥬르 뱅뱅사거리점", "멜론", "오아시스", "에이림커피", "카페S", "원할매이모네닭한마리",
    "20년전통영이네떡볶이", "영이네추억의떡볶이", "비채", "한강수", "고래똥", "두메 도곡점", "썸",
      "샵5827 에스프레소",
    "메종보탄"
]
 
# CSV 저장 경로 설정
output_folder = '200m이내_가게_리뷰'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 크롬 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# 가게별로 웹브라우저를 열고 리뷰 추출 후 닫기
for store_name in store_names:
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    driver.set_window_size(1920,1080)
    driver.get("https://map.kakao.com/")

    # 코치 레이어 이미지 닫기
    try:
        coach_layer_image = driver.find_element(By.CSS_SELECTOR, "body > div.coach_layer.coach_layer_type1 > div > div > img")
        driver.execute_script("arguments[0].click();", coach_layer_image)
        print("코치 레이어 이미지를 클릭했습니다.")
    except NoSuchElementException:
        print("코치 레이어 이미지가 존재하지 않습니다. 다음 단계로 진행합니다.")

    # 검색창 찾기 및 검색어 입력
    search_box = driver.find_element(By.ID, "search.keyword.query")
    search_box.send_keys(store_name)
    search_box.send_keys(Keys.ENTER)

    # 첫 번째 검색 결과 클릭
    time.sleep(1)
    first_result = driver.find_element(By.CSS_SELECTOR, '.placelist .PlaceItem .link_name')
    first_result.click()

    # 업종 정보 추출
    try:
        subcategory = driver.find_element(By.CSS_SELECTOR, 'span[data-id="subcategory"]')
        business_category = subcategory.text
        print(f"{store_name}의 업종 정보를 성공적으로 추출했습니다:", business_category)
    except NoSuchElementException:
        business_category = "업종 정보 없음"
        print(f"{store_name}의 업종 정보를 찾을 수 없습니다.")

    # 첫 번째 검색 결과의 리뷰 링크 클릭
    moreview_link = driver.find_element(By.CSS_SELECTOR, 'a[data-id="numberofscore"]')
    moreview_link.click()

    # 새로운 창으로 전환
    original_window = driver.current_window_handle
    for window_handle in driver.window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            break

    # '후기 더보기' 버튼을 반복적으로 클릭하여 모든 리뷰를 로드
    while True:
        try:
            more_reviews_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="mArticle"]/div[7]/div[3]/a'))
            )
            if more_reviews_button.text == "후기 접기":
                print("모든 리뷰가 로드되었습니다.")
                break
            driver.execute_script("arguments[0].click();", more_reviews_button)
            print("후기 더보기 버튼을 클릭했습니다.")
            time.sleep(1)
        except TimeoutException:
            print("후기 더보기 버튼을 찾을 수 없거나 로드되지 않았습니다. 모든 리뷰가 이미 로드되었거나 버튼이 없습니다.")
            break
        except ElementClickInterceptedException:
            print("후기 더보기 버튼을 클릭할 수 없습니다. 다른 요소가 가로막고 있을 수 있습니다.")
            break

    print("모든 리뷰 로드 완료.")

    # 리뷰 데이터 초기화
    review_dates = []
    review_contents = []

    # 리뷰 목록 가져오기
    review_elements = driver.find_elements(By.CSS_SELECTOR, 'ul.list_evaluation li')
    for review_element in review_elements:
        try:
            date = review_element.find_element(By.CSS_SELECTOR, '.unit_info .time_write').text
            content = review_element.find_element(By.CSS_SELECTOR, '.txt_comment').text
            review_dates.append(date)
            review_contents.append(content)
        except Exception as e:
            print("리뷰 정보를 가져오는 중 문제가 발생했습니다:", e)

    # 데이터프레임으로 저장
    reviews_df = pd.DataFrame({
        '작성 날짜': review_dates,
        '리뷰 내용': review_contents,
        '업종 정보': [business_category] * len(review_dates)
    })

    # CSV 파일로 저장
    file_name = os.path.join(output_folder, f"{store_name.replace(' ', '_')}_reviews_kakao.csv")
    reviews_df.to_csv(file_name, index=False, encoding="utf-8-sig")
    print(f"{file_name} 파일로 저장되었습니다.")

    # 브라우저 닫기
    driver.quit()


코치 레이어 이미지를 클릭했습니다.
샵5827 에스프레소의 업종 정보를 성공적으로 추출했습니다: 커피전문점
후기 더보기 버튼을 찾을 수 없거나 로드되지 않았습니다. 모든 리뷰가 이미 로드되었거나 버튼이 없습니다.
모든 리뷰 로드 완료.
200m이내_가게_리뷰\샵5827_에스프레소_reviews_kakao.csv 파일로 저장되었습니다.
코치 레이어 이미지를 클릭했습니다.
메종보탄의 업종 정보를 성공적으로 추출했습니다: 와인바
후기 더보기 버튼을 찾을 수 없거나 로드되지 않았습니다. 모든 리뷰가 이미 로드되었거나 버튼이 없습니다.
모든 리뷰 로드 완료.
리뷰 정보를 가져오는 중 문제가 발생했습니다: Message: no such element: Unable to locate element: {"method":"css selector","selector":".unit_info .time_write"}
  (Session info: chrome=130.0.6723.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x007AEBD3+24307]
	(No symbol) [0x00738D74]
	(No symbol) [0x0061C323]
	(No symbol) [0x0065DC86]
	(No symbol) [0x0065DECB]
	(No symbol) [0x00653CF1]
	(No symbol) [0x0067FED4]
	(No symbol) [0x00653C14]
	(No symbol) [0x00680124]
	(No symbol) [0x00699579]
	(No symbol) [0x0067FC26]
	(No symbol) [0x0065219C]
	(No symbol) [0x0065311D]
	